<a href="https://colab.research.google.com/github/PARULCHUTANIPC/NLP/blob/TEXT-CLASSIFICATION/BRZ_word2vec_%2B_Log_Reg_%2B_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk

In [ ]:
pip install spacy

## Imports

In [1]:
import re
import nltk
from nltk.corpus import stopwords
from spacy.lang.de import German
from re import match
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow import keras
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from gensim.models import Word2Vec

## Preprocessing

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
nlp = German()

In [6]:
def articletoken(row):
    text = row['Shortname']
    doc = nlp(text)
    tokens = [token.text for token in doc]
    filtered_tokens = list(filter(lambda v: match('[^x][a-z]|[A-Z]', v), tokens))
    filtered_string = ' '.join(filtered_tokens)
    return filtered_string

In [7]:
df_train = pd.read_excel(r'AI Material Catalogue.xlsx')
df_train['Filtered_Shortname'] = df_train.apply(articletoken,axis=1)

In [8]:
df_train.head(3)

,Supplier No,Supplier Name,Material No,Shortname,Long Text,Dimension Text,Quantity Unit,Main Group No,Main Group Name,Group No,Group Name,Cost Element No,Cost Element Name,Filtered_Shortname
0,M30331,R+F,101030008000,ACO Baumrost WOTAN 50 kN 2240 x 2240 mm Typ 8...,NaN,NaN,ST,0,Tiefbau,1010,ACO Baumroste,NaN,NaN,ACO Baumrost WOTAN mm Typ quadratisch mm
1,M30331,R+F,101030030000,ACO Baumrost WOTAN 50 kN 1600 x 1600 mm Typ 3...,NaN,NaN,ST,0,Tiefbau,1010,ACO Baumroste,NaN,NaN,ACO Baumrost WOTAN mm Typ quadratisch mm
2,M30331,R+F,101030046000,ACO Baumrost WOTAN 50 kN 1920 x 1920 mm Typ 4...,NaN,NaN,ST,0,Tiefbau,1010,ACO Baumroste,NaN,NaN,ACO Baumrost WOTAN mm Typ quadratisch mm


In [9]:
stop = stopwords.words('german')
eliminate_list = stop + ['mm','Bh.','cm','kg','Kl','o.','V.','D','SW','LW','BH',
                         'BG' , 'GG' , 'A' , 'B', 'Bh' ,'H' , 'DS' , 'C', 'ID' , 'SN'
                         'L.','KL.D L.','HSD H.','H.','DB','KN','DN','KG','SR-F','SH-F','SH-M',
                         'SBR','A-D','Gr','MK','L',"K"]

In [10]:
df_train['Filtered_Shortname'] = df_train['Filtered_Shortname'].apply(lambda x:re.sub(r'[^\w\s]', " ", x))
df_train['Filtered_Shortname'] = df_train['Filtered_Shortname'].apply(lambda x:re.sub(r'\d+', ' ',x))
df_train['Nostop_Shortname'] = df_train['Filtered_Shortname'].apply(lambda x: ' '.join([word for word in x.split() if word not in eliminate_list]))
df_train['Nostop_Shortname'] = df_train['Filtered_Shortname'].apply(lambda x: re.sub(r'\b\w\b'," ",x))
#df_train['Filtered_Shortname'] = df_train['Filtered_Shortname'].apply(lambda s: re.sub(r'[^a-zA-Z\s\n\.]', ' ', s))
df_train['Nostop_Shortname'] = df_train['Nostop_Shortname'].apply(lambda s: re.sub(r"\b[a-zA-Z]\b", "", s))
df_train['Nostop_Shortname'] = df_train['Nostop_Shortname'].apply(lambda s: re.sub(r"\b[a-zA-Z][a-zA-Z]\b", "", s))
df_train= df_train.replace('       ', ' ', regex=True)
df_train= df_train.replace('  ', ' ', regex=True)

In [11]:
output = df_train.to_excel('modified_AI_Catalogue.xlsx')

## Word2Vec

In [3]:
output_df = pd.read_excel('/content/modified_AI_Catalogue.xlsx')

In [4]:
output_df1 = output_df.dropna(subset=['Cost Element No'])

In [5]:
sent = []
for i in output_df1['Nostop_Shortname']:
  sent.append(i)

labels_str = []
for i in output_df1['Cost Element No']:
  labels_str.append(str(i).split('.')[0])

In [6]:
data = {'Description': sent, 'Cost Element No': labels_str}

extracted_df = pd.DataFrame(data)

In [7]:
extracted_df['Cost Element No'].nunique()

13

In [8]:
extracted_df.sample(3)

,Description,Cost Element No
4269,Gasrohr SLM SDR Stangen,624310
3078,Reduzierflansch Bolzen Typ,620915
3386,Abwasserrohr SDR Stangen,624310


In [9]:
import ssl
try:
     _create_unverified_https_context =     ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
sentences = extracted_df['Description'].apply(nltk.word_tokenize).tolist()

In [12]:
# Train word2vec model
w2v_model = Word2Vec(sentences, min_count=1)

# Get vocabulary size
vocab_size = len(w2v_model.wv.index_to_key)
print("Vocabulary size:", vocab_size)

Vocabulary size: 637


In [13]:
# Train the word2vec model
model1 = Word2Vec(sentences, vector_size=vocab_size, window=3, min_count=1, workers=4)

# Assign labels to the model
model1.build_vocab(sentences)

# Train the model with labels
model1.train(sentences, total_examples=model1.corpus_count, epochs=30)

(360828, 626160)

In [14]:
emb = model1.wv['Beton']
emb.shape

(637,)

## Collect embeddings

In [15]:
extracted_df['tokenized_desc'] = sentences

In [16]:
model1.wv.most_similar(['Beton'])

[('CityDrän', 0.7565681338310242),
 ('Dränfuge', 0.7515472769737244),
 ('Steinstärke', 0.7502343654632568),
 ('Hochbord', 0.7467786073684692),
 ('Gehwegplatten', 0.7462906837463379),
 ('MODELL', 0.7424481511116028),
 ('Rechteckstein', 0.7423253059387207),
 ('Verbundstein', 0.7410372495651245),
 ('Rasterflor', 0.7399160861968994),
 ('verschiebes', 0.7397555112838745)]

In [55]:
# Save the trained model
model1.save('my_word2vec_model.model')

In [17]:
extracted_df.sample()

,Description,Cost Element No,tokenized_desc
3794,Frank SurePEX Gasrohr gelb SDR Rollen,624310,"[Frank, SurePEX, Gasrohr, gelb, SDR, Rollen]"


In [58]:
tokenized_desc = extracted_df['tokenized_desc']
tokenized_desc_embeddings = []

In [ ]:
X_train ---> tokenized_desc

In [19]:
import numpy as np

In [59]:
for sentence in tokenized_desc:
    sentence_embeddings = []
    for word in sentence:
        if word in model1.wv.key_to_index:
          sentence_embeddings.append(model1.wv[word])
    if sentence_embeddings:
        tokenized_desc_embeddings.append(np.mean(sentence_embeddings, axis=0))
    else:
        tokenized_desc_embeddings.append(np.zeros((model1.vector_size,)))

In [60]:
extracted_df['embeddings']=''
for i in range(len(tokenized_desc_embeddings)):
  extracted_df['embeddings'][i] = tokenized_desc_embeddings[i]

In [61]:
x = extracted_df['embeddings']
y = extracted_df['Cost Element No'] 

In [62]:
extracted_df.sample()

,Description,Cost Element No,tokenized_desc,embeddings
4328,Frank Rotationsabschneider,645005,"[Frank, Rotationsabschneider]","[-0.04321509, 0.18140915, 0.061806202, 0.04123..."


In [63]:
X = extracted_df['embeddings'].to_list()
y = extracted_df['Cost Element No'].to_list()

## Train Test Split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Logistic Regression

In [48]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [49]:
classifier.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Evaluation for Logistic Regression

In [65]:
from sklearn import metrics
y_pred_log_reg = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, y_pred_log_reg))
print(classification_report(y_test, y_pred_log_reg))

Logistic Regression Accuracy: 0.9361256544502617
              precision    recall  f1-score   support

      620310       0.90      0.74      0.81        50
      620910       1.00      1.00      1.00         4
      620915       0.99      0.99      0.99       142
      620920       0.92      0.94      0.93       222
      620925       0.66      0.59      0.62        46
      620935       0.96      0.96      0.96        75
      620945       1.00      0.25      0.40         4
      624310       0.99      1.00      0.99       211
      624315       1.00      0.95      0.98        64
      624330       0.98      1.00      0.99        89
      628005       0.00      0.00      0.00         2
      628010       0.69      0.92      0.79        36
      645005       1.00      0.90      0.95        10

    accuracy                           0.94       955
   macro avg       0.85      0.79      0.80       955
weighted avg       0.94      0.94      0.93       955



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Multiclass SVM

In [40]:
# Train the model
svm = SVC(kernel='linear', C=1, decision_function_shape='ovr')
svm.fit(X, y)

SVC(C=1, kernel='linear')

### Evaluation for SVM

In [54]:
from sklearn.metrics import classification_report

In [64]:
y_pred_svm = svm.predict(X_test)
print("SVM Accuracy:",metrics.accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.9581151832460733
              precision    recall  f1-score   support

      620310       0.93      0.80      0.86        50
      620910       1.00      1.00      1.00         4
      620915       0.99      1.00      1.00       142
      620920       0.92      0.99      0.95       222
      620925       0.84      0.57      0.68        46
      620935       1.00      0.96      0.98        75
      620945       1.00      1.00      1.00         4
      624310       1.00      1.00      1.00       211
      624315       1.00      0.98      0.99        64
      624330       1.00      1.00      1.00        89
      628005       0.00      0.00      0.00         2
      628010       0.76      0.97      0.85        36
      645005       1.00      1.00      1.00        10

    accuracy                           0.96       955
   macro avg       0.88      0.87      0.87       955
weighted avg       0.96      0.96      0.96       955



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# # Preprocess text data and map to Word2Vec embeddings
# X_train = [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'sentence']]
# y_train = [0, 1] # labels for each sentence
# X_train_embeddings = []
# for sentence in X_train:
#     sentence_embeddings = []
#     for word in sentence:
#         if word in model.wv.vocab:
#             sentence_embeddings.append(model[word])
#     if sentence_embeddings:
#         X_train_embeddings.append(np.mean(sentence_embeddings, axis=0))
#     else:
#         X_train_embeddings.append(np.zeros((model.vector_size,)))

# Train multiclass SVM on Word2Vec embeddings
clf = SVC(kernel='linear')
clf.fit(X_train_embeddings, y_train)

# Preprocess and map test data to embeddings
X_test = [['this', 'is', 'a', 'test'], ['this', 'is', 'another', 'test']]
y_test = [0, 1] # labels for each sentence
X_test_embeddings = []
for sentence in X_test:
    sentence_embeddings = []
    for word in sentence:
        if word in model.wv.vocab:
            sentence_embeddings.append(model[word])
    if sentence_embeddings:
        X_test_embeddings.append(np.mean(sentence_embeddings, axis=0))
    else:
        X_test_embeddings.append(np.zeros((model.vector_size,)))

# Evaluate performance on test set
y_pred = clf.predict(X_test_embeddings)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))